In [83]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

In [84]:
#Declaring some of the constants we will use
BASE_DIR = '/home/bangaru/Downloads/Datasets'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')#https://nlp.stanford.edu/projects/glove/
TRAIN_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/train') #http://ai.stanford.edu/~amaas/data/sentiment/
TEST_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/test') 
#Within these, I only have a pos/ and a neg/ folder containing text files 
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


In [85]:
#Function to load the data from the dataset into the notebook. Will be called twice - for train and test.
def get_data(data_dir):
    texts = []  # list of text samples
    labels_index = {'pos':1, 'neg':0}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids
    for name in sorted(os.listdir(data_dir)):
        path = os.path.join(data_dir, name)
        if os.path.isdir(path):
            label_id = labels_index[name]
            for fname in sorted(os.listdir(path)):
                    fpath = os.path.join(path, fname)
                    text = open(fpath).read()
                    texts.append(text)
                    labels.append(label_id)

 #  print('Found %s texts.' % len(texts))
   # print('Found %s unique labels.' %len(set(labels)))
    return texts, labels


In [86]:
train_texts, train_labels = get_data(TRAIN_DATA_DIR)
test_texts, test_labels = get_data(TEST_DATA_DIR)


Found 25000 texts.
Found 2 unique labels.
Found 25000 texts.
Found 2 unique labels.


In [87]:
#Just a few tests to see how the data looks like. 
#print(train_texts[0])
#print(train_labels[0])
#print(test_texts[24999])
#print(test_labels[24999])

Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.
0
I've seen this story before but my kids haven't. Boy with troubled past joins military, faces his past, falls in love and becomes a man. The mentor this time is played perfectly by Kevin Costner; An ordinary man with common everyday problems who lives an extraordinary conviction, to save lives. After losing his team he takes a teaching posi

In [88]:
#Vectorize these text samples into a 2D integer tensor using Keras Tokenizer
#Tokenizer is fit on training data only, and that is used to tokenize both train and test data.
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 88582 unique tokens.


In [89]:
#Converting this to sequences to be fed into neural network. Max seq. len is 1000 as set earlier
trainvalid_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
trainvalid_labels = to_categorical(np.asarray(train_labels))
test_labels = to_categorical(np.asarray(test_labels))
#print('Shape of train data tensor:', trainvalid_data.shape)
#print('Shape of train label tensor:', trainvalid_labels.shape)
#print('Shape of test data tensor:', test_data.shape)
#print('Shape of test label tensor:', test_labels.shape)

Shape of train data tensor: (25000, 1000)
Shape of train label tensor: (25000, 2)
Shape of test data tensor: (25000, 1000)
Shape of test label tensor: (25000, 2)


In [90]:
# split the training data into a training set and a validation set
indices = np.arange(trainvalid_data.shape[0])
np.random.shuffle(indices)
trainvalid_data = trainvalid_data[indices]
trainvalid_labels = trainvalid_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * trainvalid_data.shape[0])

x_train = trainvalid_data[:-num_validation_samples]
y_train = trainvalid_labels[:-num_validation_samples]
x_val = trainvalid_data[-num_validation_samples:]
y_val = trainvalid_labels[-num_validation_samples:]
print('Splitting the train data into train and valid is done')

Splitting the train data into train and valid is done


In [91]:
print('Preparing embedding matrix.')

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))
#print(embeddings_index["google"])


Preparing embedding matrix.
Indexing word vectors.
Found 400000 word vectors.
[ 0.22575  -0.56253  -0.05156  -0.079389  1.1876   -0.48397  -0.23342
 -0.85278   0.97495  -0.33344   0.71692   0.12644   0.31962  -1.4136
 -0.57903  -0.037286 -0.0164    0.45155  -0.29005   0.52599  -0.22534
 -0.29556  -0.032407  1.5608   -0.013499 -0.064558  0.26625   0.78595
 -0.71693  -0.93025   0.80461   1.6035   -0.30602  -0.34764   0.93872
  0.38137  -0.26743  -0.56519   0.58899  -0.14554  -0.34324   0.21291
 -0.39887   0.090042 -0.8495    0.38803  -0.5045   -0.22488   1.0644
 -0.2624    1.0334    0.06348  -0.39989   0.24236  -0.65636  -1.8107
 -0.061801  0.13795   1.1658   -0.30046  -0.50143   0.16509   0.039835
  0.62541   0.56935   0.64125   0.21308   0.30276   0.39673   0.38973
  0.28183   0.79481  -0.11962  -0.49598  -0.53195  -0.14897   0.51254
 -0.39208  -0.58535  -0.078509  0.81721  -0.73497  -0.68131   0.099243
 -0.87608   0.029632  0.33402  -0.14305   0.16964  -0.035178  0.39777
  0.71769   0

In [94]:
# prepare embedding matrix

num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
#print(num_words)
#print(len(word_index))
#print(word_index["movie"])
#print(embeddings_index.get("movie"))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

20001
88582
17
[ 0.38251    0.14821    0.60601   -0.51533    0.43992    0.061053
 -0.62716   -0.025385   0.1643    -0.22101    0.14423   -0.37213
 -0.21683   -0.08895    0.097904   0.6561     0.64455    0.47698
  0.83849    1.6486     0.88922   -0.1181    -0.012465  -0.52082
  0.77854    0.48723   -0.014991  -0.14127   -0.34747   -0.29595
  0.1028     0.57191   -0.045594   0.026443   0.53816    0.32257
  0.40788   -0.043599  -0.146     -0.48346    0.32036    0.55086
 -0.76259    0.43269    0.61753   -0.36503   -0.60599   -0.79615
  0.3929    -0.23668   -0.34719   -0.61201    0.54747    0.94812
  0.20941   -2.7771    -0.6022     0.8495     1.2549     0.017893
 -0.041901   2.1147    -0.026618  -0.28104    0.68124   -0.14165
  0.99249    0.49879   -0.67538    0.6417     0.42303   -0.27913
  0.063403   0.68909   -0.36183    0.053709  -0.16806    0.19422
 -0.47073   -0.14803   -0.58986   -0.2797     0.16792    0.10568
 -1.7601     0.0088254 -0.83326   -0.5836    -0.37079   -0.56591
  0.2069

In [96]:
print('Define a 1D CNN model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])


Training model.
(?, 1000, 100)
2
(?, 1000)


In [99]:
#Train the model. Fit to validation set. 
model.fit(x_train, y_train,
          batch_size=128,
          epochs=2, validation_data=(x_val, y_val))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
20000/20000 [==============================] - 77s 4ms/step - loss: 0.6674 - acc: 0.6144
Epoch 2/10
20000/20000 [==============================] - 71s 4ms/step - loss: 0.5050 - acc: 0.7641
Epoch 3/10
20000/20000 [==============================] - 71s 4ms/step - loss: 0.4239 - acc: 0.8105
Epoch 4/10
20000/20000 [==============================] - 71s 4ms/step - loss: 0.3718 - acc: 0.8334
Epoch 5/10
20000/20000 [==============================] - 71s 4ms/step - loss: 0.3232 - acc: 0.8595
Epoch 6/10
20000/20000 [==============================] - 71s 4ms/step - loss: 0.2801 - acc: 0.8809
Epoch 7/10
20000/20000 [==============================] - 76s 4ms/step - loss: 0.2390 - acc: 0.9007
Epoch 8/10
20000/20000 [==============================] - 72s 4ms/step - loss: 0.1896 - acc: 0.9251
Epoch 9/10
20000/20000 [==============================] - 75s 4ms/step - l